In [1]:
using Pkg
Pkg.activate("Plots")
using Plots

  Activating new project at `c:\Users\micke\Desktop\COT4501\NumericalNavier\KallynFolder\Plots`


In [ ]:
function loopingBoundary(u, v) 
    u[1, :] .= u[end, :]
    u[:, 1] .= u[:, end]
    
    v[1, :] .= v[end, :]
    v[:, 1] .= v[:, end]
end

function wallBoundary(u, v)
    u[1, :] .= 0
    u[end, :] .= 0
    u[:, 1] .= 0
    u[:, end] .= 0
    
    v[1, :] .= 0
    v[end, :] .= 0
    v[:, 1] .= 0
    v[:, end] .= 0
end

function inflowLeftOutflowRight(u, v, flow=1)
    u[1, :] .= flow
    u[end, :] .= flow
    u[:, 1] .= 0
    u[:, end] .= 0
    
    v[1, :] .= flow
    v[end, :] .= flow
    v[:, 1] .= 0
    v[:, end] .= 0
end

In [3]:
function process(boundary, u, v, nx, ny, nt=100, c=1, sigma=0.02, nu=0.01)
    dx = 2 / (nx - 1)
    dy = 2 / (ny - 1)

    dt = sigma * dx * dy / nu

    x = 0:dx:2
    y = 0:dy:2

    udiff = 1.0
    stepcount = 0
    max_cfl = 0.5

    while udiff > 0.001
        un = copy(u)
        vn = copy(v)
    
        b = build_up_b(rho, dt, dx, dy, u, v)
        p = pressure_poisson_periodic(p, dx, dy, b, nit)
    
        u[2:end-1, 2:end-1] .= un[2:end-1, 2:end-1] .-
                              un[2:end-1, 2:end-1] .* dt / dx .* 
                              (un[2:end-1, 2:end-1] - un[2:end-1, 1:end-2]) .-
                              vn[2:end-1, 2:end-1] .* dt / dy .* 
                              (un[2:end-1, 2:end-1] - un[1:end-2, 2:end-1]) .-
                              dt / (2 * rho * dx) .* 
                              (p[2:end-1, 3:end] - p[2:end-1, 1:end-2]) .+
                              nu * (dt / (dx^2) * 
                              (un[2:end-1, 3:end] - 2 .* un[2:end-1, 2:end-1] + un[2:end-1, 1:end-2]) .+
                              dt / (dy^2) * 
                              (un[3:end, 2:end-1] - 2 .* un[2:end-1, 2:end-1] + un[1:end-2, 2:end-1])) .+
                              F * dt
    
        v[2:end-1, 2:end-1] .= vn[2:end-1, 2:end-1] .-
                              un[2:end-1, 2:end-1] .* dt / dx .* 
                              (vn[2:end-1, 2:end-1] - vn[2:end-1, 1:end-2]) .-
                              vn[2:end-1, 2:end-1] .* dt / dy .* 
                              (vn[2:end-1, 2:end-1] - vn[1:end-2, 2:end-1]) .-
                              dt / (2 * rho * dy) .* 
                              (p[3:end, 2:end-1] - p[1:end-2, 2:end-1]) .+
                              nu * (dt / (dx^2) *
                              (vn[2:end-1, 3:end] - 2 .* vn[2:end-1, 2:end-1] + vn[2:end-1, 1:end-2]) .+
                              dt / (dy^2) *
                              (vn[3:end, 2:end-1] - 2 .* vn[2:end-1, 2:end-1] + vn[1:end-2, 2:end-1]))
    
        boundary(x, y)
    
        udiff = abs.(sum(u) - sum(un)) / sum(u)
        stepcount += 1
    
        cfl = calculate_cfl(u, v, dt, dx, dy)
        if cfl > max_cfl
            println("CFL number is too large: $cfl.")
            break
        end
    end
end

process (generic function with 5 methods)

In [2]:
# uniform density, looping boundary conditions



0-dimensional Array{Float64, 0}:
1.0